In [1]:
import plotly.graph_objects as go
import plotly.express as px
from shapely.geometry import Polygon,MultiPolygon,Point
import numpy as np
import math
import miscellaneous_functions

In [40]:
def create_polygon(x_min,y_min,x_max,y_max):
    x_coords = [x_min,x_min,x_max,x_max,x_min]
    y_coords = [y_min,y_max,y_max,y_min,y_min]
    polygon_coords = tuple(zip(x_coords,y_coords))
    polygon_obj = Polygon(polygon_coords)
    return polygon_coords,x_coords,y_coords,polygon_obj

In [58]:
p1 = create_polygon(10,100,40,200)
p2 = create_polygon(50,100,70,200)
p3 = create_polygon(10,200,40,300)
p4 = create_polygon(50,50,100,250)
p5 = create_polygon(80,100,110,200)
p6 = create_polygon(160,350,170,400)
p7 = create_polygon(150,300,180,500)
p8 = create_polygon(160,350,170,400)
p9 = create_polygon(20,301,40,350)

In [59]:
polygon_set = [p1,p2,p3,p4,p5,p6,p7,p8,p9]

In [60]:
fig = go.Figure()

for i in range(len(polygon_set)):
    fig.add_trace(go.Scatter(
        x=polygon_set[i][1],
        y=polygon_set[i][2],
        name = "p"+str(i+1) # legend entry 
    ))
    
fig.show()

In [61]:
# # DC check
# p1[3].disjoint(p2[3])
# p1[3].disjoint(p3[3])
# p4[3].disjoint(p5[3])

In [62]:
# #EC check
# p1[3].intersection(p3[3]).geom_type == 'LineString'

In [63]:
# # EQ check
# (p6[3].intersection(p8[3]) == p6[3]) and (p6[3].intersection(p8[3]) == p8[3])

In [64]:
#TPP check
def tangential_touch_check(p1,p2):
    '''
    input: two polygons
    output: True/False
    '''
    TPP_flag = False
    for i in range(4):
        if p1.bounds[i]==p2.bounds[i]:
            TPP_flag = True
            break
    return TPP_flag

In [65]:
# RCC-8 code
def rcc_8_check(p1,p2):
    '''
    input: two polygons
    output: rcc-8 representation
    '''
    representation = "ND"
    if p1.disjoint(p2):
        if p1.distance(p2)<=1:
            representation = "EC"
        else:
            representation = "DC"
    else:
        if p1.intersection(p2).geom_type == "LineString":
            representation = "EC"
        elif ((p1.intersection(p2)==p1) and (p1.intersection(p2)==p2)):
            representation = "EQ"
        elif p1.within(p2):
            representation = "TPP" if tangential_touch_check(p1,p2) else "NTPP"
        elif p2.within(p1):
            representation = "TPPi" if tangential_touch_check(p1,p2) else "NTPPi"
        elif ((p1.intersection(p2)!=p1) and (p1.intersection(p2)!=p2)):
            representation = "PO"
#             overlap_polygon = p1.intersection(p2)
#             smallest_side = min(quardratic_equation_solver(1,-overlap_polygon.length/2,overlap_polygon.area))
#             #the adjustment is done to account for noice
#             if smallest_side <=1:
#                 representation = "EC"
#             else:
#                 representation = "PO"
        else:
            print(p1.intersection(p2))
            
    return representation

In [66]:
for Pi in range(len(polygon_set)):
    for Pj in range(len(polygon_set)):
        rep = rcc_8_check(polygon_set[Pi][3],polygon_set[Pj][3])
        print(Pi+1,Pj+1,rep)

1 1 EQ
1 2 DC
1 3 EC
1 4 DC
1 5 DC
1 6 DC
1 7 DC
1 8 DC
1 9 DC
2 1 DC
2 2 EQ
2 3 DC
2 4 TPP
2 5 DC
2 6 DC
2 7 DC
2 8 DC
2 9 DC
3 1 EC
3 2 DC
3 3 EQ
3 4 DC
3 5 DC
3 6 DC
3 7 DC
3 8 DC
3 9 DC
4 1 DC
4 2 TPPi
4 3 DC
4 4 EQ
4 5 PO
4 6 DC
4 7 DC
4 8 DC
4 9 DC
5 1 DC
5 2 DC
5 3 DC
5 4 PO
5 5 EQ
5 6 DC
5 7 DC
5 8 DC
5 9 DC
6 1 DC
6 2 DC
6 3 DC
6 4 DC
6 5 DC
6 6 EQ
6 7 NTPP
6 8 EQ
6 9 DC
7 1 DC
7 2 DC
7 3 DC
7 4 DC
7 5 DC
7 6 NTPPi
7 7 EQ
7 8 NTPPi
7 9 DC
8 1 DC
8 2 DC
8 3 DC
8 4 DC
8 5 DC
8 6 EQ
8 7 NTPP
8 8 EQ
8 9 DC
9 1 DC
9 2 DC
9 3 DC
9 4 DC
9 5 DC
9 6 DC
9 7 DC
9 8 DC
9 9 EQ


In [12]:
#QDC code

In [13]:
#touches relation - to
p1[3].disjoint(p2[3])

#far/near relation
threshold = 70

#creating the bigger polygon for relation check
p0xmin, p0ymin, p0xmax, p0ymax = (np.array(p1[3].bounds)+(-threshold,-threshold,threshold,threshold))
p0 = create_polygon(p0xmin, p0ymin, p0xmax, p0ymax)

#if false: nr , true: fr
p0[3].disjoint(p2[3])

False

In [14]:
p1[3].disjoint(p2[3]) == False

False

In [15]:
def qdc_check(p1,p2,threshold_x =70, threshold_y = 70):
    '''
    input: two polygons, threshold to determine far/near
    output: qdc representation
    '''
    representation = "ND"
    if p1.disjoint(p2) == False:
        representation = "to"
    elif p1.disjoint(p2):
        #create the bigger polygon for relation check
        p0xmin, p0ymin, p0xmax, p0ymax = (np.array(p1.bounds)+(-threshold_x,-threshold_y,threshold_x,threshold_y))
        p0 = create_polygon(p0xmin, p0ymin, p0xmax, p0ymax)[3]   
        representation = "fr" if p0.disjoint(p2) else "nr"
    else:
        print(p1.intersection(p2))
        print(p0.intersection(p2))
    return representation

In [16]:
for Pi in range(len(polygon_set)):
    for Pj in range(len(polygon_set)):
        rep = qdc_check(polygon_set[Pi][3],polygon_set[Pj][3])
        print(Pi+1,Pj+1,rep)

1 1 to
1 2 nr
1 3 to
1 4 nr
1 5 nr
1 6 fr
1 7 fr
1 8 fr
2 1 nr
2 2 to
2 3 nr
2 4 to
2 5 nr
2 6 fr
2 7 fr
2 8 fr
3 1 to
3 2 nr
3 3 to
3 4 nr
3 5 nr
3 6 fr
3 7 fr
3 8 fr
4 1 nr
4 2 to
4 3 nr
4 4 to
4 5 to
4 6 fr
4 7 nr
4 8 fr
5 1 nr
5 2 nr
5 3 nr
5 4 to
5 5 to
5 6 fr
5 7 fr
5 8 fr
6 1 fr
6 2 fr
6 3 fr
6 4 fr
6 5 fr
6 6 to
6 7 to
6 8 to
7 1 fr
7 2 fr
7 3 fr
7 4 nr
7 5 fr
7 6 to
7 7 to
7 8 to
8 1 fr
8 2 fr
8 3 fr
8 4 fr
8 5 fr
8 6 to
8 7 to
8 8 to


In [17]:
# STAR code

In [18]:
c_x, c_y = p1[3].centroid.x,p1[3].centroid.y

In [19]:
hypotenuse_len = 100

In [20]:
triangle_ad_factor_cos = hypotenuse_len*np.cos(67.5*np.pi/180)
triangle_ad_factor_sin = hypotenuse_len*np.sin(67.5*np.pi/180)

In [21]:
triangle_ad_factor_sin

92.38795325112868

In [22]:
t0,t1,t2,t3,t4,t5,t6,t7 = 0,0,0,0,0,0,0,0

In [23]:
def adjust_point(p,x_adjust,y_adjust):
    '''
    input: reference point, x-adjust point, y-adjust point
    output: adjusted point
    '''
    p_adj = p[0]+x_adjust,p[1]+y_adjust
    
    return p_adj

In [24]:
def triangles_for_STAR(p=(0,0),hypotenuse_len=500):
    '''
    input: reference point, hypotenuse length
    output: coordinates of the STAR ref traingle 
    '''
    cos_factor = hypotenuse_len*np.cos(67.5*np.pi/180)
    sin_factor = hypotenuse_len*np.sin(67.5*np.pi/180)
    
    t1 = adjust_point(p,cos_factor,sin_factor)
    t2 = adjust_point(p,sin_factor,cos_factor)
    t3 = adjust_point(p,sin_factor,-cos_factor)
    t4 = adjust_point(p,cos_factor,-sin_factor)
    t5 = adjust_point(p,-cos_factor,-sin_factor)
    t6 = adjust_point(p,-sin_factor,-cos_factor)
    t7 = adjust_point(p,-sin_factor,cos_factor)
    t8 = adjust_point(p,-cos_factor,sin_factor)
    
    STAR_rep = ["tp","tr","rt","br","bt","bl","lt","tl"]
    t = [t1,t2,t3,t4,t5,t6,t7,t8]
        
    polygon_list = []
    
    polygon_list.append([STAR_rep[0],Polygon([p,t1,t8])])
    for i in range(1,8):
        p0 = Polygon([p,t[i-1],t[i]])
        polygon_list.append([STAR_rep[i],p0])
        
    return polygon_list

In [25]:
def STAR_check(p1=(0,0),p=(0,0),hypotenuse_len=500):
    '''
    input: point to check, point of reference, hypotenuse length
    output: STAR representation
    '''
    triangle_rep = triangles_for_STAR(p,hypotenuse_len)
    
    representation = "ND"
    for i in range(8):
        if Point(p1).disjoint(triangle_rep[i][1])==False:
            representation = triangle_rep[i][0]
            break
            
    return representation

In [26]:
STAR_check((100,-100),(0,0),1500)

'br'

In [94]:
for Pi in range(len(polygon_set)):
    for Pj in range(len(polygon_set)):
        ref_point = (polygon_set[Pi][3].centroid.x,polygon_set[Pi][3].centroid.y)
        check_obj = (polygon_set[Pj][3].centroid.x,polygon_set[Pj][3].centroid.y)
        rep = STAR_check(check_obj,ref_point,1000)
        
        print(Pj+1,Pi+1,rep)
        #print(check_obj,ref_point,rep)

1 1 tp
2 1 rt
3 1 tp
4 1 rt
5 1 rt
6 1 tr
7 1 tr
8 1 tr
1 2 lt
2 2 tp
3 2 tp
4 2 rt
5 2 rt
6 2 tr
7 2 tr
8 2 tr
1 3 bt
2 3 bt
3 3 tp
4 3 br
5 3 br
6 3 tr
7 3 tr
8 3 tr
1 4 lt
2 4 lt
3 4 tl
4 4 tp
5 4 rt
6 4 tp
7 4 tp
8 4 tp
1 5 lt
2 5 lt
3 5 tl
4 5 lt
5 5 tp
6 5 tp
7 5 tp
8 5 tp
1 6 bl
2 6 bl
3 6 bl
4 6 bt
5 6 bt
6 6 tp
7 6 tp
8 6 tp
1 7 bl
2 7 bl
3 7 bl
4 7 bt
5 7 bt
6 7 bt
7 7 tp
8 7 bt
1 8 bl
2 8 bl
3 8 bl
4 8 bt
5 8 bt
6 8 tp
7 8 tp
8 8 tp


In [59]:
p2_triangle = triangles_for_STAR(p=(60,150),hypotenuse_len=500)
p2_triangle

[['tp', <shapely.geometry.polygon.Polygon at 0x1d269f15e48>],
 ['tr', <shapely.geometry.polygon.Polygon at 0x1d269f15e08>],
 ['rt', <shapely.geometry.polygon.Polygon at 0x1d269f15b88>],
 ['br', <shapely.geometry.polygon.Polygon at 0x1d269f15f08>],
 ['bt', <shapely.geometry.polygon.Polygon at 0x1d269f15d88>],
 ['bl', <shapely.geometry.polygon.Polygon at 0x1d269f1db08>],
 ['lt', <shapely.geometry.polygon.Polygon at 0x1d269f1df08>],
 ['tl', <shapely.geometry.polygon.Polygon at 0x1d269f1de88>]]

In [93]:
fig = go.Figure()

for i in range(len(polygon_set)):
    fig.add_trace(go.Scatter(
        x=polygon_set[i][1],
        y=polygon_set[i][2],
        name = "p"+str(i+1) # legend entry 
    ))
    
fig.add_trace(go.Scatter(
    x = [25],
    y = [250],
    name = "c3"
    ))
    
fig.add_trace(go.Scatter(
    x = [60],
    y = [150],
    name = "c2"
    ))

for i in range(len(p2_triangle)):
    x_triangle = []
    y_triangle = []
    for j in range(4):
        x_triangle.append(p2_triangle[i][1].exterior.coords.xy[0][j])
        y_triangle.append(p2_triangle[i][1].exterior.coords.xy[1][j])
        
    fig.add_trace(go.Scatter(
        x = x_triangle,
        y = y_triangle,
        name = str(p2_triangle[i][0])
        ))
    
    
    
fig.show()

In [52]:
triangles_for_STAR(p=(60,150),hypotenuse_len=500)[0][1].bounds

(-131.34171618254493, 150.0, 251.34171618254493, 611.9397662556433)

In [76]:
triangles_for_STAR(p=(60,150),hypotenuse_len=500)[0][1].exterior.coords.xy[0]

array('d', [60.0, 251.34171618254493, -131.34171618254493, 60.0])

In [57]:
p2_triangle = triangles_for_STAR(p=(60,150),hypotenuse_len=500)

[['tp', <shapely.geometry.polygon.Polygon at 0x1d269f41348>],
 ['tr', <shapely.geometry.polygon.Polygon at 0x1d269f41448>],
 ['rt', <shapely.geometry.polygon.Polygon at 0x1d269f41688>],
 ['br', <shapely.geometry.polygon.Polygon at 0x1d269f41608>],
 ['bt', <shapely.geometry.polygon.Polygon at 0x1d269f41648>],
 ['bl', <shapely.geometry.polygon.Polygon at 0x1d269f414c8>],
 ['lt', <shapely.geometry.polygon.Polygon at 0x1d269f41208>],
 ['tl', <shapely.geometry.polygon.Polygon at 0x1d269f41288>]]